In [ ]:
%env SERVER_URL   hub.gke.mybinder.org
%env PROXY_SUBURL  proxy/7474/browser/

In [ ]:
!pip3 install ipython-cypher neo4j py2neo >/dev/null

In [ ]:
!curl http://localhost:7474/

In [ ]:
%load_ext cypher

In [ ]:
%%cypher 
CREATE (p1:Person {name:'Bob'}),
       (p2:Person {name:'Alice'}   )
return p1, p2